# Applied Data Science Capstone Project - Week 2, part 1

Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = requests.get(url)
# Store the contents of the website inside the variable "d"
d = lh.fromstring(webpage.content)
# Parse data that are stored between <tr>..</tr> of HTML inside the variable "data"
data = d.xpath('//tr')
# Store in a list "col_list" the name of each table column
col_list = []
for idx, t in zip(range(1, 4), data[0]):
    name = str(t.text_content()).strip()
    print(f"Column {idx}: {name}")
    col_list.append((name, []))

Column 1: Postal code
Column 2: Borough
Column 3: Neighborhood


In [3]:
# Extract only the data pertaining to the table of postal codes
flag = 1
for _ in range(1, len(data)):
    row = data[_]
    # If the row does not have size = 3, the //tr data does not correspond to the table row, so do not extract any data
    if len(row) != 3:
        flag = 0
    elif flag == 1:
        col = 0
        for t in row.iterchildren():
            temp = t.text_content() 
            # Check if the row is empty
            if col > 0:
            # Convert any numerical value to integers
                try:
                    temp = int(data)
                except TypeError:
                    pass
            # Append the data to the empty list of the i'th column
            col_list[col][1].append(str(temp).strip())
            # Increment i for the next column
            col += 1

In [4]:
# Creation of a dataframe
data_Dict = {title:column for (title, column) in col_list}
df = pd.DataFrame(data_Dict)

In [5]:
# Moving the "Postal code" column in first position
df = df[['Postal code', 'Borough', 'Neighborhood']]
# Removing all the rows that do not have a borough assigned 
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
# Removing all the rows that do not have a postal code
df.drop(df.index[df['Postal code'] == ''], inplace = True)
# Resetting the dataframe index to zero
df.reset_index(drop=True, inplace=True)
df = df.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df['Neighborhood'].replace('\n','/', regex=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Saving the dataframe as a .csv file
df.to_csv(r'Canada_postal_codes.csv')
